# MAT 381E Homework 2

## Question 1

For this question we are going to use [an NBA dataset](https://sports-statistics.com/sports-data/nba-basketball-datasets-csv-files/). 

1. Get the data from the season 2018-2019 as a pandas dataframe. Explain each column in the dataset.
2. Write a function that takes the `GAME_ID` as the parameter and returns the full dataset for a given match. Test your function on 5 random matches.
3. Write a function that takes the `GAME_ID` as the parameter and returns a dictionary of (key,value) pairs where key is a player name and value is how many times that specific player appears in `PLAYER1_NAME`,`PLAYER2_NAME`, or `PLAYER3_NAME` columns.
4. Plot the results of Step 3 as a bar chart sorted from the most active player to the least active player for 5 random games.
3. Write a function that takes the `GAME_ID` as the parameter and returns a list of pairs of player names where the pair interacted during the game more than once. Test your function on 5 random matches.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import networkx as nx
import seaborn as sns

from urllib.request import Request, urlopen
from urllib.parse import urlencode
from collections import Counter
from zipfile import ZipFile
from io import BytesIO
from xmltodict import parse

c:\Users\ozden\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with urlopen('https://sports-statistics.com/database/basketball-data/nba/2018-19_pbp.csv') as url:
    nba_18 = pd.read_csv(url)
pd.set_option('display.max_rows', 25)    
nba_18.head(100)

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
0,0,0,12,2,21800794,NaN,NaN,12:00,1,0.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM
1,1,0,10,4,21800794,Jump Ball Ayton vs. Faried: Tip to Okobo,NaN,12:00,1,4.0,...,1629059,Elie Okobo,PHX,Phoenix,1.610613e+09,Suns,NaN,NaN,NaN,9:11 PM
2,2,1,5,7,21800794,Jackson Bad Pass Turnover (P1.T1),NaN,11:39,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden STEAL (1 STL),9:11 PM
3,3,6,1,9,21800794,NaN,NaN,11:31,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM
4,4,2,6,10,21800794,Bridges S.FOUL (P1.T1) (E.Dalen),NaN,11:31,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:12 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,11,3,136,21800794,NaN,NaN,1:57,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,24 - 21,-3,Harden Free Throw 1 of 2 (11 PTS),9:31 PM
96,96,12,3,137,21800794,NaN,NaN,1:57,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,25 - 21,-4,Harden Free Throw 2 of 2 (12 PTS),9:31 PM
97,97,2,6,138,21800794,NaN,NaN,1:39,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green S.FOUL (P1.PN) (E.Dalen),9:32 PM
98,98,11,3,140,21800794,Holmes Free Throw 1 of 2 (1 PTS),NaN,1:39,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,25 - 22,-3,NaN,9:32 PM


In [3]:
nba_18["PERSON2TYPE"].unique()

array([0, 5, 4, 7], dtype=int64)

In [4]:
nba_18.loc[nba_18["PERSON1TYPE"] ==5]

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
3,3,6,1,9,21800794,NaN,NaN,11:31,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM
5,5,10,3,12,21800794,NaN,NaN,11:31,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Harden Free Throw 1 of 1,9:12 PM
7,7,1,6,15,21800794,NaN,NaN,11:13,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden P.FOUL (P1.T1) (T.Maddox),9:13 PM
9,9,50,1,19,21800794,NaN,NaN,11:00,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,4 - 0,-4,Ennis III 1' Running Dunk (2 PTS),9:13 PM
10,10,1,6,20,21800794,NaN,NaN,10:43,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden P.FOUL (P2.T2) (E.Dalen),9:13 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582457,425,0,8,624,21801037,NaN,NaN,0:05,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Nwaba FOR Zizic,10:47 PM
582458,426,28,6,626,21801037,NaN,NaN,0:04,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Osman Personal Take Foul (P2.PN) (D.Taylor),10:48 PM
582460,428,0,8,629,21801037,NaN,NaN,0:04,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Zizic FOR Nwaba,10:49 PM
582464,432,0,8,635,21801037,NaN,NaN,0:04,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Clarkson FOR Zizic,10:50 PM


In [5]:
nba_18["HOMEDESCRIPTION"].dropna()

1            Jump Ball Ayton vs. Faried: Tip to Okobo
2                   Jackson Bad Pass Turnover (P1.T1)
4                    Bridges S.FOUL (P1.T1) (E.Dalen)
6                        Booker REBOUND (Off:0 Def:1)
8                     Okobo Bad Pass Turnover (P1.T2)
                             ...                     
582456    MAVERICKS Timeout: Regular (Full 5 Short 0)
582459               Harris Free Throw 1 of 2 (5 PTS)
582461                 SUB: Finney-Smith FOR Nowitzki
582462               Harris Free Throw 2 of 2 (6 PTS)
582466                   Kleber REBOUND (Off:5 Def:7)
Name: HOMEDESCRIPTION, Length: 302186, dtype: object

In [6]:
nba_18.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582468 entries, 0 to 582467
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 582468 non-null  int64  
 1   EVENTMSGACTIONTYPE         582468 non-null  int64  
 2   EVENTMSGTYPE               582468 non-null  int64  
 3   EVENTNUM                   582468 non-null  int64  
 4   GAME_ID                    582468 non-null  int64  
 5   HOMEDESCRIPTION            302186 non-null  object 
 6   NEUTRALDESCRIPTION         0 non-null       float64
 7   PCTIMESTRING               582468 non-null  object 
 8   PERIOD                     582468 non-null  int64  
 9   PERSON1TYPE                581974 non-null  float64
 10  PERSON2TYPE                582468 non-null  int64  
 11  PERSON3TYPE                582468 non-null  int64  
 12  PLAYER1_ID                 582468 non-null  int64  
 13  PLAYER1_NAME               53

## NBA 2018-2019 Data Set
1. EVENTMSGACTIONTYPE : type of a action that happened in game (foul, free throw....)
2. EVENTMSGTYPE : type of a event in game (point, rebound, free throw, miss..)
3. EVENTNUM	: ? (connected between other event columns i guess)
4. GAME_ID	: nba game id 
5. HOMEDESCRIPTION : describtion of event that happens to home team (?)
6. NEUTRALDESCRIPTION	: all NaN 
7. PCTIMESTRING	: time left for that period
8. PERIOD	: number of a period in game 
9. PERSON1TYPE	: player role in NBA game like point guard, shooting guard, forward, center (?)
10. PERSON2TYPE	: player role in NBA game like point guard, shooting guard, forward, center (?)? 
11. PERSON3TYPE	: player role in NBA game like point guard, shooting guard, forward, center (?)
12. PLAYER1_ID	: spesific number for player 1
13. PLAYER1_NAME	: player 1 name 
14. PLAYER1_TEAM_ABBREVIATION	: abbreviation of team's name of player 1 
15. PLAYER1_TEAM_CITY	: player1's team city
16. PLAYER1_TEAM_ID	: player 1's spesific team number
17. PLAYER1_TEAM_NICKNAME	: player 1 team nickname
18. PLAYER2_ID	: spesific number for player 2
19. PLAYER2_NAME	: player 2 name
20. PLAYER2_TEAM_ABBREVIATION	: abbreviation of team's name of player 2 
21. PLAYER2_TEAM_CITY	: player2's team city
22. PLAYER2_TEAM_ID	: player 2's spesific team number
23. PLAYER2_TEAM_NICKNAME	: player 2 team nickname
24. PLAYER3_ID	: spesific number for player 3
25. PLAYER3_NAME	: 
26. PLAYER3_TEAM_ABBREVIATION	
27. PLAYER3_TEAM_CITY	
28. PLAYER3_TEAM_ID	
29. PLAYER3_TEAM_NICKNAME	
30. SCORE	
31. SCOREMARGIN	
32. VISITORDESCRIPTION	
33. WCTIMESTRING

In [7]:
def gameID(gameNumber):
    df = nba_18[nba_18["GAME_ID"] == gameNumber]
    return df

In [8]:
gameID(21800799)

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
155273,0,0,12,2,21800799,NaN,NaN,12:00,1,0.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:42 PM
155274,1,0,10,4,21800799,Jump Ball Jordan vs. Drummond: Tip to Bullock,NaN,12:00,1,4.0,...,203493,Reggie Bullock,DET,Detroit,1.610613e+09,Pistons,NaN,NaN,NaN,7:42 PM
155275,2,52,1,7,21800799,NaN,NaN,11:41,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Drummond 1' Alley Oop Dunk (2 PTS) (Jackson 1 ...,7:42 PM
155276,3,1,6,9,21800799,NaN,NaN,11:27,1,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jackson P.FOUL (P1.T1) (M.Myers),7:43 PM
155277,4,3,2,11,21800799,MISS Vonleh 3' Hook Shot,NaN,11:13,1,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:43 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155705,432,0,4,610,21800799,NaN,NaN,0:33,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Drummond REBOUND (Off:4 Def:12),9:54 PM
155706,433,7,1,611,21800799,NaN,NaN,0:19,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,105 - 92,-13,Drummond 1' Dunk (17 PTS) (Jackson 7 AST),9:55 PM
155707,434,1,2,613,21800799,MISS Thomas 25' 3PT Jump Shot,NaN,0:12,4,4.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:55 PM
155708,435,0,4,614,21800799,NaN,NaN,0:10,4,5.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bullock REBOUND (Off:0 Def:7),9:55 PM


In [9]:
nba_18.groupby(["GAME_ID","PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"]).count()

Unnamed: 0  \
GAME_ID  PLAYER1_NAME    PLAYER2_NAME        PLAYER3_NAME                  
21800001 Al Horford      Joel Embiid         Ben Simmons               1   
         Jaylen Brown    Dario Saric         Al Horford                1   
21800002 Damian Jones    Steven Adams        Draymond Green            1   
         Draymond Green  Dennis Schroder     Kevin Durant              1   
         Kevon Looney    Jerami Grant        Kevin Durant              1   
...                                                                  ...   
21801228 Nikola Jokic    Gorgui Dieng        Jamal Murray              1   
         Will Barton     Gorgui Dieng        Tyus Jones                1   
21801229 Ivica Zubac     Ekpe Udoh           Grayson Allen             1   
                         Tony Bradley        Jerome Robinson           1   
21801230 Skal Labissiere Willie Cauley-Stein Jake Layman               1   

                                                              EVENTMSGACTIONTYPE  \
GAME_ID  PLAYER1_NAME    PLAYER2_NAME        PLAYER3_NAME                          
21800001 Al Horford      Joel Embiid         Ben Simmons                       1   
         Jaylen Brown    Dario Saric         Al Horford                        1   
21800002 Damian Jones    Steven Adams        Draymond Green                    1   
         Draymond Green  Dennis Schroder     Kevin Durant                      1   
         Kevon Looney    Jerami Grant        Kevin Durant                      1   
...                                                                          ...   
21801228 Nikola Jokic    Gorgui Dieng        Jamal Murray                      1   
         Will Barton     Gorgui Dieng        Tyus Jones                        1   
21801229 Ivica Zubac     Ekpe Udoh           Grayson Allen                     1   
                         Tony Bradley        Jerome Robinson                   1   
21801230 Skal Labissiere Willie Cauley-Stein Jake Layman                       1   

                                                              EVENTMSGTYPE  \
GAME_ID  PLAYER1_NAME    PLAYER2_NAME        PLAYER3_NAME                    
21800001 Al Horford      Joel Embiid         Ben Simmons                 1   
         Jaylen Brown    Dario Saric         Al Horford                  1   
21800002 Damian Jones    Steven Adams        Draymond Green              1   
         Draymond Green  Dennis Schroder     Kevin Durant                1   
         Kevon Looney    Jerami Grant        Kevin Durant                1   
...                                                                    ...   
21801228 Nikola Jokic    Gorgui Dieng        Jamal Murray                1   
         Will Barton     Gorgui Dieng        Tyus Jones                  1   
21801229 Ivica Zubac     Ekpe Udoh           Grayson Allen               1   
                         Tony Bradley        Jerome Robinson             1   
21801230 Skal Labissiere Willie Cauley-Stein Jake Layman                 1   

                                                              EVENTNUM  \
GAME_ID  PLAYER1_NAME    PLAYER2_NAME        PLAYER3_NAME                
21800001 Al Horford      Joel Embiid         Ben Simmons             1   
         Jaylen Brown    Dario Saric         Al Horford              1   
21800002 Damian Jones    Steven Adams        Draymond Green          1   
         Draymond Green  Dennis Schroder     Kevin Durant            1   
         Kevon Looney    Jerami Grant        Kevin Durant            1   
...                                                                ...   
21801228 Nikola Jokic    Gorgui Dieng        Jamal Murray            1   
         Will Barton     Gorgui Dieng        Tyus Jones              1   
21801229 Ivica Zubac     Ekpe Udoh           Grayson Allen           1   
                         Tony Bradley        Jerome Robinson         1   
21801230 Skal Labissiere Willie Cauley-Stein Jake Layman             1   

  

In [107]:
def playerApp(gameNumber):
    df = nba_18[nba_18["GAME_ID"] == gameNumber]
    df_01 = df.groupby(nba_18["PLAYER1_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()
    df_02 = df.groupby(nba_18["PLAYER2_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()
    df_03 = df.groupby(nba_18["PLAYER3_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()
    return df_01.add(df_02,fill_value=0).add(df_03,fill_value=0)

In [109]:
playerApp(21800002)

C:\Users\ozden\AppData\Local\Temp\ipykernel_12292\60087556.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_01 = df.groupby(nba_18["PLAYER1_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()
C:\Users\ozden\AppData\Local\Temp\ipykernel_12292\60087556.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_02 = df.groupby(nba_18["PLAYER2_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()
C:\Users\ozden\AppData\Local\Temp\ipykernel_12292\60087556.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_03 = df.groupby(nba_18["PLAYER3_NAME"].rename("player_name"))["PLAYER1_NAME","PLAYER2_NAME","PLAYER3_NAME"].count()


,PLAYER1_NAME,PLAYER2_NAME,PLAYER3_NAME
player_name,,,
Alex Abrines,20.0,13.0,0.0
Alfonzo McKinnie,2.0,1.0,0.0
Andre Iguodala,8.0,5.0,0.0
Damian Jones,30.0,20.0,5.0
Dennis Schroder,61.0,28.0,3.0
Draymond Green,46.0,26.0,3.0
Hamidou Diallo,14.0,10.0,1.0
Jeff Hornacek,1.0,0.0,0.0
Jerami Grant,28.0,14.0,5.0


## Question 2

For this question we are going to use [a sample](https://raw.githubusercontent.com/arch-raven/spotify-recsys/main/spotify_million_playlist/dataset/data/mpd.slice.0-999.json) from [Spottify Million Playlist Dataset](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge).

1. Ingest the data as a python dictionary using the [json](https://docs.python.org/3/library/json.html) library.
2. Construct a complete list of all artists appearing in the dictionary you obtained in step 1. Then using [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) function count how many times each artist appears.
3. Filter out those artists that appear fewer than 100 times, and then sort the remaining from the most popular to least popular.
4. Plot the result you obtain from Step 3 as a bar chart.
5. Repeat Steps 2-4 for the Songs instead of Artists by filtering out the songs that appear 25 times or fewer.

## Question 3

For this question we are going to use [Wordbank Data](https://data.worldbank.org/). I would suggest [wbgapi](https://pypi.org/project/wbgapi/) library.

1. Write a function that takes a country name and a list of indicators and returns a pandas data frame with the corresponding data.
2. Get the data for Turkey for the following indicators:
   * [Poverty headcount ratio at national poverty lines (SI.POV.NAHC)](https://data.worldbank.org/indicator/SI.POV.NAHC)
   * [Current health expenditure per capita (SH.XPD.CHEX.PC.CD)](https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD)
   * [Labor force participation rate, female (% of female population ages 15+) (SL.TLF.CACT.FE.NE.ZS)](https://data.worldbank.org/indicator/SL.TLF.CACT.FE.NE.ZS)
   * [Mortality rate, under-5 (per 1,000 live births) (SH.DYN.MORT)](https://data.worldbank.org/indicator/SH.DYN.MORT)
   * [Literacy rate, adult female (% of females ages 15 and above) (SE.ADT.LITR.FE.ZS)](https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS)
3. Write a function that draws a line plot for a given list of indicators against time (i.e. the x-axis will be the time) AFTER it drops the years where data is not available. Draw a line plot for Turkey for all of the indicators I listed above.
4. Write a function that takes two indicators `id1` and `id2` and does a scatter plot of `id1` against `id2` after it drops the years where data is not available for one of the indicators. Your plot must have labels on the x- and the y-axes to indicate which indicators were used. Then draw a scatter plot (10 in total) for every pair of indicators I listed above.